# Dynamic Queries for **Gans** Cities Database

In [1]:
import pandas as pd
import sqlalchemy
from mining_functions import pop_scrape_api, forecasting, icao_airport_codes, airports, arrrivals_by_airport, push_population, push_airports, push_cities_airports
from connection import connection_cloud_local, connection_local, connection_cloud_online
connection = connection_local()

## Population per City

In [6]:
# Load City Data
cities_sql = pd.read_sql("cities", con=connection)
cities_sql

,city_id,city_name,country,longitude,latitude
0,1,Berlin,Germany,13.405000,52.520000
1,2,Hamburg,Germany,10.000000,53.549999
2,3,Munich,Germany,11.575000,48.137501
3,4,Paris,France,2.352222,48.856667
4,5,Leipzig,Germany,12.375000,51.340000
5,6,Halle (Saale),Germany,11.969722,51.482777
6,7,Dresden,Germany,13.740000,51.049999
7,8,Chemnitz,Germany,12.916667,50.833332
8,9,Cologne,Germany,6.952778,50.936390


In [7]:
population_df = pop_scrape_api(cities_sql)
population_df

,city_id,population,date_fetched
0,1,3644826,2024-04-15 12:18:27
1,2,1841179,2024-04-15 12:18:28
2,3,1471508,2024-04-15 12:18:30
3,4,11020000,2024-04-15 12:18:31
4,5,542529,2024-04-15 12:18:32
5,6,239257,2024-04-15 12:18:33
6,7,555805,2024-04-15 12:18:35
7,8,245403,2024-04-15 12:18:36
8,9,1085664,2024-04-15 12:18:37


update SQL table

In [8]:
push_population(population_df)

'population data was updated\n'

## Weather Forecast per City

In [32]:
# Load City Data
cities_sql = pd.read_sql("cities", con=connection)

In [33]:
forecast_df = forecasting(cities_sql)
forecast_df

,city_id,forecast_time,outlook,temp,feels_like,wind_speed,rain_3h,date_fetched
0,1,2024-04-15 12:00:00,light rain,13.18,12.07,5.63,0.41,2024-04-15 12:29:44
1,1,2024-04-15 15:00:00,light rain,13.00,11.92,4.95,0.73,2024-04-15 12:29:44
2,1,2024-04-15 18:00:00,light rain,12.65,11.51,2.85,1.05,2024-04-15 12:29:44
3,1,2024-04-15 21:00:00,light rain,7.84,4.10,7.21,2.04,2024-04-15 12:29:44
4,1,2024-04-16 00:00:00,light rain,6.15,1.85,7.41,0.26,2024-04-15 12:29:44
...,...,...,...,...,...,...,...,...
355,9,2024-04-19 21:00:00,light rain,6.28,2.78,5.35,0.25,2024-04-15 12:29:45
356,9,2024-04-20 00:00:00,light rain,5.57,2.31,4.47,0.13,2024-04-15 12:29:45
357,9,2024-04-20 03:00:00,overcast clouds,5.93,2.97,4.06,0.00,2024-04-15 12:29:45
358,9,2024-04-20 06:00:00,overcast clouds,6.02,3.63,3.14,0.00,2024-04-15 12:29:45


Push to SQL:

In [34]:
forecast_df.to_sql('forecast',
                  if_exists='append',
                  con=connection,
                  index=False)

360

## Airport per City

In [2]:
# Load City Data
cities_sql = pd.read_sql("cities", con=connection)

In [3]:
# Call the function for all cities
cities_airports_df = icao_airport_codes(cities_sql)
cities_airports_df

,icao_code,city_id
0,EDDB,1
1,EDDH,2
2,EDDM,3
3,LFPB,4
4,LFPO,4
5,LFPG,4
6,EDDP,5
7,EDDP,6
8,EDDC,7
9,EDDK,9


before pushing to SQL, create further table:

## Airport Info

In [25]:
airports_df = airports(cities_airports_df)
airports_df

,icao_code,iata_code,full_name,latitude,longitude,country_code
0,EDDB,BER,Berlin Brandenburg,52.35139,13.493889,DE
1,EDDH,HAM,Hamburg,53.63040,9.988229,DE
2,EDDM,MUC,Munich,48.35380,11.786100,DE
3,LFPB,LBG,Paris -Le Bourget,48.96940,2.441390,FR
4,LFPO,ORY,Paris -Orly,48.72530,2.359440,FR
5,LFPG,CDG,Paris Charles de Gaulle,49.01280,2.549999,FR
6,EDDP,LEJ,Leipzig Halle,51.43240,12.241600,DE
7,EDDC,DRS,Dresden,51.13280,13.767200,DE
8,EDDK,CGN,Cologne Bonn,50.86590,7.142739,DE
9,EDDL,DUS,Duesseldorf Düsseldorf,51.28950,6.766779,DE


push *airports* to sql:

In [26]:
push_airports(airports_df)

airports table was updated



push *cities_airports* to sql:

In [5]:
push_cities_airports(cities_airports_df)

cities_airports table updated



## Flight Arrivals per City

In [8]:
# Load City Data
airports_df = pd.read_sql("airports", con=connection)
airports_df

,icao_code,iata_code,full_name,latitude,longitude,country_code
0,EDDB,BER,Berlin Brandenburg,52.351391,13.493889,DE
1,EDDH,HAM,Hamburg,53.630402,9.988229,DE
2,EDDM,MUC,Munich,48.353802,11.786100,DE
3,EDDP,LEJ,Leipzig Halle,51.432400,12.241600,DE
4,LFPB,LBG,Paris -Le Bourget,48.969398,2.441390,FR
5,LFPG,CDG,Paris Charles de Gaulle,49.012798,2.549999,FR
6,LFPO,ORY,Paris -Orly,48.725300,2.359440,FR


In [9]:
arrivals_df = arrrivals_by_airport(airports_df)
arrivals_df

**ERROR**
Airport: LFPB
Code: 204
Reason: No Content


,arrival_airport_icao,departure_airport_icao,departure_airport_name,departure_scheduledTime_local,arrival_scheduledTime_local,arrival_revisedTime_local,aircraft_model,airline_name,date_fetched
0,EDDB,HEMA,Marsa Alam,2024-04-11 01:20+02:00,2024-04-11 06:20+02:00,2024-04-11 06:20+02:00,Airbus A320,Air Cairo,2024-04-10 16:52:44
1,EDDB,LTBJ,İzmir,2024-04-11 04:25+03:00,2024-04-11 06:30+02:00,2024-04-11 06:30+02:00,Boeing 737-800 (winglets),Sun Express,2024-04-10 16:52:44
2,EDDB,ZBAA,Beijing,2024-04-11 02:40+08:00,2024-04-11 06:45+02:00,2024-04-11 06:45+02:00,Airbus A330-300,Hainan,2024-04-10 16:52:44
3,EDDB,KEWR,Newark,2024-04-10 17:05-04:00,2024-04-11 07:15+02:00,2024-04-11 07:15+02:00,Boeing 767-400(ER),United,2024-04-10 16:52:44
4,EDDB,NaN,Chisinau,NaN,2024-04-11 07:20+02:00,2024-04-11 07:20+02:00,Boeing 737-800,Fly One,2024-04-10 16:52:44
...,...,...,...,...,...,...,...,...,...
1881,LFPO,EGKK,London,2024-04-11 19:40+01:00,2024-04-11 22:00+02:00,NaN,Airbus A321,Vueling,2024-04-10 16:52:56
1882,LFPO,LPPT,Lisbon,2024-04-11 19:35+01:00,2024-04-11 23:00+02:00,NaN,Airbus A320,TAP Air Portugal,2024-04-10 16:52:56
1883,LFPO,LEMG,Málaga,2024-04-11 19:15+02:00,2024-04-11 21:45+02:00,NaN,Airbus A320,Vueling,2024-04-10 16:52:56
1884,LFPO,LIMC,Milan,2024-04-11 20:10+02:00,2024-04-11 21:45+02:00,NaN,Airbus A321,Vueling,2024-04-10 16:52:56


In [10]:
arrivals_df.value_counts("arrival_airport_icao")

arrival_airport_icao
LFPG    601
EDDM    450
LFPO    330
EDDB    326
EDDH    162
EDDP     17
Name: count, dtype: int64

This result is reasonable. 

The API reports *no content* for Airport *LFPB*. This Airport is not used by travelers (Wikipedia).

Push arrivals to SQL

In [ ]:
arrivals_df.to_sql('arrivals',
                  if_exists='append',
                  con=connection,
                  index=False)

1886